## <center> Аттестация. Практическое задание в Jupyter

В данном блоке вам предстоит решить задачу, которую автор когда-то решал на одном из своих собеседований.
Вам предоставлена таблица с данными (скачать файл можно на платформе) о ежедневной выработке энергии солнечными батареями. Информация в ней содержится в двух столбцах: 
* date - дата наблюдения
* cum_power - накопленная (кумулятивная) выработка энергии на указанный день

То есть перед вами классический пример временного ряда: числовой показатель, зависящий от времени. Давайте с ним поработаем.


In [6]:
import pandas as pd
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go 

import io
import zipfile
import requests  

plt.style.use('default')    # корректный вывод графиков в темной теме 
# fig.show(renderer='notebook') #включение анимации

r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/7e342d3b6dc19a057cc4dc535a8ce591/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/solarpower.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('data')

#https://lms.skillfactory.ru/assets/courseware/v1/7e342d3b6dc19a057cc4dc535a8ce591/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/solarpower.zip

1. В нашем временном ряде есть несколько пропусков, они обозначены числом -1. Заполните пропуски средним арифметическим между двумя соседними наблюдениями: </p>
    $$x^*_i  = \frac{x_{i-1} + x_{i+1}}{2}$$ 
    
Совет: проще всего будет сделать это, используя цикл по индексам элементов столбца cum_power, но вы можете предложить более изящное решение.


In [7]:
solarpower = pd.read_csv('data\solarpower.csv', sep=',')
display(solarpower.head())

solarpower['date'] =pd.to_datetime(solarpower['date'])
display(solarpower.info())
display(solarpower.describe())

,date,cum_power
0,2017-01-01,20111.0
1,2017-01-02,20112.0
2,2017-01-03,20115.0
3,2017-01-04,20117.0
4,2017-01-05,20119.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       664 non-null    datetime64[ns]
 1   cum_power  664 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 10.5 KB


None

,cum_power
count,664.000000
mean,23775.573795
std,2636.399680
min,-1.000000
25%,22043.750000
50%,24036.000000
75%,25320.750000
max,28120.000000


In [11]:
# %%timeit

def fill_empty_spaces(df_collumn, empty_spaces = [ -1], how = 'add'):
    ''' 
    данная функция будет заполнять пропуски, обозначенные в списке [ 0,'0', 'NaN', ' ', 'np.NaN'] в переданной в данную функцию колонке,
    на среднее значение предшествующего и следующего значения, 
    так же доступен функционал добавления/замены  дефолтного списка своих значений how = 'add'(default) / 'replace'
    '''
    
    lst_of_looked = [ 0,'0', 'NaN', ' ', 'np.NaN'] # наш список вариантов пропусков
    #  проверка какой список будем использовать
    if how == 'add': #  если 'add' то добавляем
        lst_of_looked.extend(empty_spaces)
    elif how == 'replace': #  если 'replace' то заменяем
        lst_of_looked = empty_spaces
    else: # 3й вариант указываем на ошибку
        raise(Exception, r"Wrong argument 'how', should be 'add' or 'replace'")

    data_to_check = df_collumn.tolist()
    data_to_return=[]

    for ind, val in enumerate(data_to_check):
        if val not in lst_of_looked:
            data_to_return.append(val)
        else:
            data_to_return.append((data_to_check[ind-1] + data_to_check[ind+1])/2)
      
    return data_to_return
    
  
solarpower['cum_power'] = fill_empty_spaces(solarpower['cum_power'], [ -1], how='add')    

2. Теперь, когда таблица заполнена, создайте столбец day_power - ежедневный прирост выработки солнечной энергии.

    После чего создайте данные о ежемесячной выработке энергии на каждый год. 
    
Совет: для того, чтобы выделить из даты год и месяц проще всего воспользоваться методом аксессора dt.to_period('M'), который позволяет выделять периоды, в данном случае 'М' - месяц.

In [21]:
solarpower['day_power'] = (solarpower['cum_power']).diff()
solarpower['month'] =solarpower['date'].dt.to_period('M').astype('str')
df = solarpower.groupby('month')['day_power'].agg(['sum']).reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   month   22 non-null     object 
 1   sum     22 non-null     float64
dtypes: float64(1), object(1)
memory usage: 480.0+ bytes


3. Постройте график временного ряда - ежемесячной выработки солнечной энергии. Не забудьте подписать оси и сам график.
Укажите в какие сезоны наблюдается подъем, а в какие спад?

In [27]:
fig = px.line(df, x='month', y='sum', width= 1000)
fig.update_layout(title='Monthly energy production',
                   xaxis_title='Month',
                   yaxis_title='Energy production')
fig.update_xaxes(dtick='M1', tickangle=-45)

fig.show()